# 0A - Fourier Transfrom
---

In [1]:
import numpy as np
np.set_printoptions(linewidth=np.inf)
np.set_printoptions(threshold=np.inf)


---

>$\textbf{Definition 1.}$
>
>복소수 영역 ($\mathbb{C}$) 에서 관측된 $N$ 개의 데이터가 있다고 해보자:
>
>\begin{equation*}
>    \{ x_n \} := x_0, x_1, \dots, x_{N-1}
>\end{equation*}
>
>이 때 Discrete Fourier Transformation ($\textsf{DFT}$) 의 결과집합인 $\{ X_k \} := X_0, X_1, \dots, X_{N-1}$ 은 다음과 같이 정의된다:
>
>\begin{align*}
>    X_k = \sum_{n=0}^{N-1} x_n \cdot \exp{\left(-i 2\pi n \frac{k}{N}\right)}
>\end{align*}

In [14]:
def DFT(x):
    N = len(x)
    X = np.zeros(N, dtype=complex)

    for k in range(N):
        X_k = 0
        for n, x_n in enumerate(x):
            X_k += x_n * np.exp(-1j * 2 * np.pi * n * k / N)
        X[k] = X_k

    return X

In [23]:
x = [1, 2, 3, 4]
X = DFT(x)
print("Our DFT      : ", X)
print("Original FFT : ", np.fft.fft(x))

Our DFT      :  [10.+0.00000000e+00j -2.+2.00000000e+00j -2.-9.79717439e-16j -2.-2.00000000e+00j]
Original FFT :  [10.+0.j -2.+2.j -2.+0.j -2.-2.j]


>$\textbf{Definition 2.}$
>
>$\textsf{DFT}$ 의 결과집합인 $\{ X_k \}$ 가 주어졌다고 할 때,
>이에 대한 Inverse Discrete Fourier Transform ($\textsf{IDFT}$) 의 결과 집합 $\{x_n\}$ 은 다음과 같이 정의된다:
>
>\begin{align*}
>    x_n = \frac{1}{N} \sum_{k=0}^{N-1} X_k \cdot \exp{\left( i 2 \pi n \frac{k}{N} \right)}.
>\end{align*}


In [24]:
def IDFT(X):
    N = len(X)
    x = np.zeros(N, dtype=complex)

    for n in range(N):
        x_n = 0
        for k, X_k in enumerate(X):
            x_n += X_k * np.exp(1j * 2 * np.pi * n * k / N)
        x[n] = (1/N) * x_n

    return x

In [22]:
x_ = IDFT(X)
print("Our DFT      : ", x_.real)
print("Original Msg : ", x)

Our DFT      :  [1. 2. 3. 4.]
Original Msg :  [1, 2, 3, 4]


---

## Real and Imaginary part

다음의 표는 time domain 과 frequency domain 에서의 real, imag part 에 대한 연산의 정의를 나타내고 있다.


\begin{array}{c|c|c}
    \textbf{Property}        & \textbf{Time domain $x_n$}    & \textbf{Freq domain $X_k$} \\ \hline
    \text{Real part in time} & \textsf{Re}(x_n)              & \frac{1}{2}(X_k + X^*_{N-k}) \\
    \text{Imag part in time} & \textsf{Im}(x_n)              & \frac{1}{2}(X_k - X^*_{N-k}) \\
    \text{Real part in freq} & \frac{1}{2}(x_n + x^*_{N-n})  & \textsf{Re}(X_k)             \\
    \text{Imag part in freq} & \frac{1}{2}(x_n - x^*_{N-n})  & \textsf{Im}(X_k)             \\
\end{array}

추가적으로 참고할 것은 time domain 에서 conjugation 은 다음과 같은 성질을 갖는다는 것이다.

\begin{equation*}
    \text{If, } \textsf{DFT}(\{x_n\})_k = X_k \text{, then } \textsf{DFT}(\{x^*_n\})_k = X^*_{N-k}
\end{equation*}


In [48]:
x = np.array([1 + 1j, 2 + 2j, 3 + 3j, 4 + 4j])
x_conj = np.conjugate(x)

X      = DFT(x)
X_conj = DFT(x_conj)
X_tmp  = np.conjugate(X)
X_tmp  = np.concatenate((X_tmp[:1], X_tmp[:0:-1])) # X^*_{N-k}

print(X_conj)
print(X_tmp)

[ 1.0000000e+01-1.00000000e+01j -8.8817842e-16+4.00000000e+00j -2.0000000e+00+2.00000000e+00j -4.0000000e+00-2.22044605e-15j]
[ 1.00000000e+01-1.0000000e+01j  2.22044605e-15+4.0000000e+00j -2.00000000e+00+2.0000000e+00j -4.00000000e+00+8.8817842e-16j]


---
## Shift Theorem

\begin{align*}
    &\text{If, } \textsf{DFT}(\{x_n\})_k = X_k \\
    &\text{then, } \textsf{DFT}\left( \left\{
        x_n \cdot \exp(i 2 \pi nm / N)
    \right\} \right)_k = X_{k-m} \\
    &\text{and, } \textsf{DFT}(\{ x_{n-m}\})_k = X_k \cdot \exp(- 2i\pi km/N)
\end{align*}

In [104]:
m = 1
N = 4
x = [1, 2, 3, 4]
X = DFT(x)
X_roll = np.roll(X, m)

x_ = [elem * np.exp(2 * 1j * np.pi * n * m / N)  for n, elem in enumerate(x)]
X_ = DFT(x_)

print(X_roll)
print(X_)

x_roll = np.roll(x, m)

X__  = DFT(x_roll)
X___ = [elem * np.exp(-2j * np.pi * k * m / N) for k, elem in enumerate(X)]

print(X__)
print(X___)

[-2.-2.00000000e+00j 10.+0.00000000e+00j -2.+2.00000000e+00j -2.-9.79717439e-16j]
[-2.-2.00000000e+00j 10.+0.00000000e+00j -2.+2.00000000e+00j -2.-9.79717439e-16j]
[10.+0.00000000e+00j  2.+2.00000000e+00j  2.-7.34788079e-16j  2.-2.00000000e+00j]
[(10+0j), (1.9999999999999993+2.0000000000000004j), (2+1.2246467991473533e-15j), (2.0000000000000013-1.9999999999999978j)]


---
## Circular convolution theorem and cross-correlation theorem

\begin{align*}
    x \ast y_N = \textsf{IDFT}(\textsf{DFT}(\{x_N\}) \cdot \textsf{DFT}(\{y_N\}))
\end{align*}

In [111]:
x = [1, 2, 3, 4]
y = [5 ,6, 7, 8]

X = DFT(x)
Y = DFT(y)

O = [_x * _y for _x, _y in zip(X,Y)]
o = IDFT(O)
print(o.real) # result via (I)DFT

poly_x = np.poly1d(x)
poly_y = np.poly1d(y)
div = np.zeros(4 + 1)
div[0] = -1
div[-1] = 1
poly_div = np.poly1d(div)

poly_mul = np.polymul(poly_x, poly_y)
q, r = np.polydiv(poly_mul, poly_div)
print(r.coef[::-1]) # result via cyclic convolution

[66. 68. 66. 60.]
[66. 68. 66. 60.]
